In [46]:
#imports

import shlex, subprocess
import os , sys
import time

import numpy as np
from numpy import array
from numpy import argmax
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow import keras
import keras_tuner

from Bio import SeqIO

import matplotlib.pyplot as plt
import seaborn as sns    
        

In [47]:
#one hot encoding
def fasta_to_onehotencode(seq) : 
    #values = list(seq)
    #values = np.array(values)
    base2int = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    # label = int(line.strip()[1])
    
    
    sequence = seq 
    # Encode sequence bases as integers, i.e. A as 0, C as 1, etc.
    sequence_int = [base2int.get(base, 9999) for base in sequence]
    
    sequence_onehot = tf.one_hot(sequence_int, depth=4)
    
        
    return sequence_onehot

In [48]:
def fastatoarray(fasta_sequences) :
    seq_array = np.zeros((1,400,4)) 
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        new_sequence = fasta_to_onehotencode(sequence) #onehotencode(sequence)
        new_sequence = np.expand_dims(new_sequence, axis =0)
        seq_array = np.vstack((seq_array,new_sequence))
    seq_array = np.delete(seq_array, 0, 0) #to remove the first array of zeros 
    return seq_array  

In [49]:
_start_time = time.time()
#change a bit this function or cite 
def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

In [50]:
def prepare_dataset(path, current_folder) : 
    print('Starting preparation of the dataset for %s' %current_folder)
    
    folder = current_folder
    directory2 = os.listdir(path+ '/' + folder)
    seq_list_positive = np.zeros((1,400,4)) 
    seq_list_negative = np.zeros((1,400,4)) 
    seq_list_positive_test = np.zeros((1,400,4)) 
    seq_list_negative_test = np.zeros((1,400,4)) 

    for folder2 in directory2 :
        directory3 = os.listdir(path+ '/' + folder + '/' + folder2)

        if folder2 != "fold-4": #to be split into training and validation

            seq_list_positive_file = np.zeros((1,400,4)) 
            seq_list_negative_file = np.zeros((1,400,4)) 

            for file in directory3 :

                if "fasta" in file :
                    fasta_sequences = SeqIO.parse(open(path+ '/' + folder + '/' + folder2+ '/' + file),'fasta')
                    seq_array = fastatoarray(fasta_sequences)

                    if "positive" in file : 
                        seq_list_positive_file = np.vstack((seq_list_positive_file, seq_array))


                    if "negative-1" in file: # 2 neg: 1 pos with 'negative' --> 1 neg : 1 pos 'negative-1'
                        seq_list_negative_file = np.vstack((seq_list_negative_file, seq_array))

            seq_list_negative_file = np.delete(seq_list_negative_file, 0, 0)
            seq_list_positive_file = np.delete(seq_list_positive_file, 0, 0)

            seq_list_positive = np.vstack((seq_list_positive, seq_list_positive_file))
            seq_list_negative = np.vstack((seq_list_negative, seq_list_negative_file))



        if folder2 == "fold-4": #for testing

                seq_list_positive_test_file = np.zeros((1,400,4)) 
                seq_list_negative_test_file = np.zeros((1,400,4)) 

                for file in directory3 :

                    if "fasta" in file :
                        fasta_sequences = SeqIO.parse(open(path+ '/' + folder + '/' + folder2+ '/' + file),'fasta')
                        seq_array = fastatoarray(fasta_sequences)

                        if "positive" in file : 
                            seq_list_positive_test_file = np.vstack((seq_list_positive_test_file, seq_array))


                        if "negative-1" in file: 
                            seq_list_negative_test_file = np.vstack((seq_list_negative_test_file, seq_array))

                seq_list_negative_test_file = np.delete(seq_list_negative_test_file, 0, 0)
                seq_list_positive_test_file = np.delete(seq_list_positive_test_file, 0, 0)

                seq_list_positive_test = np.vstack((seq_list_positive_test, seq_list_positive_test_file))
                seq_list_negative_test = np.vstack((seq_list_negative_test, seq_list_negative_test_file))

                seq_list_negative_test = np.delete(seq_list_negative_test, 0, 0)
                seq_list_positive_test = np.delete(seq_list_positive_test, 0, 0)  



    seq_list_negative = np.delete(seq_list_negative, 0, 0)
    seq_list_positive = np.delete(seq_list_positive, 0, 0)
    
    
    #prep labels
    labels_positive = np.ones((np.shape(seq_list_positive)[0], 1))

    labels_negative = np.zeros((np.shape(seq_list_negative)[0], 1))

    labels_positive_test = np.ones((np.shape(seq_list_positive_test)[0], 1))

    labels_negative_test = np.zeros((np.shape(seq_list_negative_test)[0], 1))
    print('Shape of labels: \n-positive : ',np.shape(labels_negative),'\n-negative : ', np.shape(labels_negative),'\n-positive validation : ',np.shape(labels_positive_test),'\n-negative validation : ',np.shape(labels_negative_test))
    
    
    #merging datasets
    x = np.vstack((seq_list_positive, seq_list_negative))

    x_test = np.vstack((seq_list_positive_test, seq_list_negative_test))

    y = np.vstack((labels_positive, labels_negative))

    y_test = np.vstack((labels_positive_test, labels_negative_test))
    
    #creating the validation set

    #parameters 
    test_size = 0.2

    #test set

    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=test_size, shuffle= True)
    x_test, y_test = shuffle(x_test, y_test, random_state=0)

    y_train = y_train.astype("float32")#actually useful?
    y_val =y_val.astype("float32")
    y_test = y_test.astype("float32")

    x_train = x_train.astype("float32")
    x_val = x_val.astype("float32")
    x_test =x_test.astype("float32")



    print('Shape of datasets: \n-training set : ',np.shape(x_train),'\n-validation set : ',np.shape(x_val),'\n-testing set : ', np.shape(x_test))
    
    #checking class ditribution in the whole dataset and training set

    print('Label frequencies among the dataset %s' %current_folder)
    plt.hist(y)
    plt.xticks(range(2))
    plt.title('Label Frequency')
    plt.show()

    plt.hist(y_train)
    plt.xticks(range(2))
    plt.title('Label Frequency training set')
    plt.show()

    plt.hist(y_val)
    plt.xticks(range(2))
    plt.title('Label Frequency validation set')
    plt.show()
    return x_train ,y_train ,x_val ,y_val ,x_test ,y_test

In [51]:
#tuning of the model architecture 
def build_model(hp):
    
    
    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(curve= 'ROC',name='auroc'),
      keras.metrics.AUC(curve = 'PR', name='auprc')  
    ]

   
    #the first layer should be a convolutional layer
    #conv-maxpooling-dropout-conv-maxpooling-dropout-flatten-(dense-dropout)-dense --> this is the pysster architecture 
    #how long should the kernel be? 
    #the parameters are now the default ones from pysster
    
    input_shape = (400, 4)
    
    
    #hyperparameter search 
    
    hp_filters = hp.Int('filters', min_value = 10, max_value = 60, step = 10)
    hp_kernel_size = hp.Int('kernel_size', min_value = 10, max_value = 60, step = 5)
    hp_pool_size = hp.Int('pool_size', min_value = 1, max_value = 10, step = 1)
    hp_strides = hp.Int('strides', min_value = 1, max_value = 10, step = 1)
    hp_learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    hp_padding = hp.Choice('padding', ['valid','same'])
    hp_kernel_initializer = hp.Choice('initiaizer', ['random_normal','random_uniform', 'he_normal', 'glorot_normal'])
    hp_activation=hp.Choice("activation", ["relu", "tanh"])
    
    
    
    # create model
    
        
    model = keras.Sequential()
    
    model.add(keras.layers.InputLayer(input_shape= input_shape))
    #model.add(keras.layers.Dropout(0.3)) -- seems better without this step
    
    model.add(keras.layers.Conv1D(filters=hp_filters,kernel_size=hp_kernel_size,
              kernel_initializer=hp_kernel_initializer, 
              activation=hp_activation,input_shape=input_shape))
    model.add(keras.layers.MaxPooling1D(pool_size=hp_pool_size, strides=hp_strides, padding=hp_padding))#valid or same?  
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Conv1D(filters=hp_filters,kernel_size=hp_kernel_size,
              kernel_initializer= hp_kernel_initializer, 
              activation=hp_activation ,input_shape=input_shape ))
    model.add(keras.layers.MaxPooling1D(pool_size=hp_pool_size, strides=hp_strides, padding=hp_padding))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Flatten())#without parameters
    
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
        layers.Dense(
            # Tune number of units separately.
            units=hp.Int(f"units_{i}", min_value=32, max_value=1024, step=32),
            activation = hp_activation
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation=hp_activation))

    model.add(keras.layers.Dropout(0.6))

    model.add(keras.layers.Dense(1, activation='sigmoid'))#what should be the output shape? 1? 

    
    #many models train better when reducing the learning rate gradually -- not in this case!! see model2
    #lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                                                                 # 0.001,
                                                                 # decay_steps=50*1000,
                                                                 # decay_rate=1,
                                                                  #staircase=False)
    
    
    # Compile model

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(hp_learning_rate), metrics=METRICS)
    
    
    return model
    
    

In [52]:
def prepare_best_model(x_train ,y_train ,x_val ,y_val ,x_test ,y_test, folder) : 
    
    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(curve= 'ROC',name='auroc'),
      keras.metrics.AUC(curve = 'PR', name='auprc')  
    ]
    
    my_callbacks1 = [
        tf.keras.callbacks.EarlyStopping(patience=2), #better to reduce to 2 for the search 
        tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        ]
    
        my_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2), #better to reduce to 2 for the search 
        tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        ]


    model2 = build_model(keras_tuner.HyperParameters())

    print(model2.summary())

    
    tuner = keras_tuner.RandomSearch(
        hypermodel=build_model,
        #objective=keras_tuner.Objective("val_auroc", direction="max"), #val_loss , val_accuracy .... 77
        objective = keras_tuner.Objective("val_loss", direction="min"),
        max_trials=150,
        executions_per_trial=1, #probably better to change to 1 and increase the max_trials
        overwrite=True,
        directory="my_dir",
        project_name="models",
    )

    print(tuner.search_space_summary())


    tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val), callbacks = my_callbacks1)
    
    # Get the top 2 models.
    models2 = tuner.get_best_models(num_models=4)
    best_model2 = models2[0]
    # Build the model.
    # Needed for `Sequential` without specified `input_shape`.
    best_model2.build(input_shape=(None, 400, 4))
    best_model2.summary()

    
    best_hps = tuner.get_best_hyperparameters(5)
    # Build the model with the best hp.
    model2 = build_model(best_hps[0])
    # Fit with the entire dataset.

    history = model2.fit(x_train, y_train, epochs=200, validation_data= (x_val,y_val), callbacks = my_callbacks)
    
    
    
    # list all data in history --> add name of the rbp!!!!

    print(history.history.keys())
    
    plt.title('Best model evaluation for %s' %folder)
    
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for auroc 
    plt.plot(history.history['auroc'])
    plt.plot(history.history['val_auroc'])
    plt.plot(history.history['auprc'])
    plt.plot(history.history['val_auprc'])
    plt.title('model auroc and auprc')
    plt.ylabel('auroc and auprc')
    plt.xlabel('epoch')
    plt.legend(['auroc_train', 'auroc_test','auprc_train', 'auprc_test'], loc='upper left')
    plt.show()
    
    model2.save('best_models/best_model4_%s' %folder) 

In [53]:
def testing_model(path , current_folder, x_test, y_test) : 
    reconstructed_model = keras.models.load_model(path)
    
    y_pred = reconstructed_model.predict(x_test)

    threshold = 0.5

    y_pred2 = np.where(y_pred > threshold, 1,0)
    
    #Get the confusion matrix model1

    cf_matrix = confusion_matrix(y_test, y_pred2)

    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ['{0:0.0f}'.format(value) for value in
                    cf_matrix.flatten()]
    group_percentages = ['{0:.2%}'.format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
    
    #saving the true and predicted labels
    
    df= pd.DataFrame(y_pred, columns = ['Predictions'])
    df['True'] = y_test
    
    fpr, tpr, thresholds = metrics.roc_curve(df['True'], df['Predictions'], drop_intermediate=False)
    
    roc_auc = metrics.roc_auc_score(df['True'], df['Predictions'])
    

    # Generate figure
    fig = pl.figure(figsize=(14,7))
    ax = fig.add_subplot(121)

    pl.plot(fpr, tpr, label=" (AUC=%.2f)" % roc_auc)
    ax.plot([0,1], [0,1], color="grey",label="Random Classifier",linestyle="--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_ylim(0,1)
    ax.set_xlim(0,1)
    ax.grid(color="#CCCCCC")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    #rbp = str(file).split('.')[0]
    pt.title('AUROC %s' % current_folder)
    pl.legend()
    pl.tight_layout()
    

    #saving the predictions (0,1) in a csv file -- in probabilities 

    df.to_csv("predictions/%s_4.csv" % current_folder)
    
    print("Prediction saved.")



In [54]:
#change to function the majority of the notebook !! 

path = '/lustre/groups/crna01/workspace/sofia/gencode/processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


tic()

for folder in directory1:
    if folder == 'AARS_K562':
    
        #preparation of the dataset from the respective folder 
        x_train ,y_train ,x_val ,y_val ,x_test ,y_test = prepare_dataset(path, folder)

        #prepare and save the best model in the respective folder 
        prepare_best_model(x_train ,y_train ,x_val ,y_val ,x_test ,y_test, folder) 

        #evaluation and storage of the predicted labels
        model_path = ('best_models/best_model4_%s' % folder)
        testing_model(model_path , folder, x_test, y_test)
tac()
    

Trial 128 Complete [00h 00m 00s]

Best val_loss So Far: 0.46264326572418213
Total elapsed time: 00h 20m 18s

Search: Running Trial #129

Value             |Best Value So Far |Hyperparameter
10                |50                |filters
40                |55                |kernel_size
5                 |9                 |pool_size
10                |4                 |strides
0.002051          |0.00031558        |lr
valid             |valid             |padding
random_uniform    |random_uniform    |initiaizer
relu              |tanh              |activation
2                 |2                 |num_layers
736               |288               |units_0
True              |False             |dropout
32                |64                |units_1
352               |352               |units_2



Traceback (most recent call last):
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 250, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 215, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 286, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 212, in _build_and_fit_model
    model = self._try_build(hp)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/eng

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 250, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 215, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 286, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 212, in _build_and_fit_model
    model = self._try_build(hp)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 154, in _try_build
    model = self._build_hypermodel(hp)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 145, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_260924/303801283.py", line 53, in build_model
    model.add(keras.layers.Conv1D(filters=hp_filters,kernel_size=hp_kernel_size,
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/icb/sofia.martello/miniconda3/envs/my-thesis-env/lib/python3.10/site-packages/keras/layers/convolutional/base_conv.py", line 347, in compute_output_shape
    raise ValueError(
ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d_1. Consider increasing the input size. Received input shape [None, 36, 10] which would produce output shape with a zero or negative value in a dimension.


45/45 [==============================] - 1s 3ms/step


In [18]:
model_path = ('best_models/best_model2_%s' % folder)
testing_model(model_path , folder, x_test, y_test)